In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
import agate

In [22]:
%%bash
## download the mixedbev file
curl -L -O https://comptroller.texas.gov/auto-data/odc/MIXEDBEV_02_2017.CSV

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2565k  100 2565k    0     0  2789k      0 --:--:-- --:--:-- --:--:-- 2949k


In [40]:
%%bash
head -n 5 MIXEDBEV_02_2017.CSV

"MB821424    ","ABI-HAUS                      ","959 N 2ND ST                  ","ABILENE             ","TX","79601","221","          ","2016/12", 000000637.97
"MB638028    ","ABILENE BEEHIVE INC           ","442 CEDAR ST STE A            ","ABILENE             ","TX","79601","221","          ","2017/01", 000002557.12
"MB543114    ","ABILENE BOWLING LANES INC     ","279 RUIDOSA AVE               ","ABILENE             ","TX","79605","221","          ","2017/01", 000000287.49
"MB933130    ","ABILENE CABARET LLC           ","1918 BUTTERNUT ST             ","ABILENE             ","TX","79602","221","          ","2017/01", 000000988.04
"N 037863    ","ABILENE COUNTRY CLUB          ","4039 S TREADAWAY BLVD         ","ABILENE             ","TX","79602","221","          ","2017/01", 000002068.82


In [35]:
file = 'MIXEDBEV_02_2017.CSV'
column_names = [
    'TABC Permit Number',
    'Trade Name',
    'Location Address',
    'Location City',
    'Location State',
    'Location Zip Code',
    'Location County Code',
    'Blank',
    'Report Period',
    'Report Tax'
]
specified_types = {
    'Location Zip Code': agate.Text(),
    'Location County Code': agate.Text()
}

In [36]:
mixbev = agate.Table.from_csv(file, column_names, encoding='iso-8859-1', column_types=specified_types)

In [37]:
print(mixbev)

| column               | data_type |
| -------------------- | --------- |
| TABC Permit Number   | Text      |
| Trade Name           | Text      |
| Location Address     | Text      |
| Location City        | Text      |
| Location State       | Text      |
| Location Zip Code    | Text      |
| Location County Code | Text      |
| Blank                | Boolean   |
| Report Period        | Text      |
| Report Tax           | Number    |



In [42]:
mixbev.limit(5).print_table()

| TABC Permit Number | Trade Name           | Location Address     | Location City        | Location State | Location Zip Code | ... |
| ------------------ | -------------------- | -------------------- | -------------------- | -------------- | ----------------- | --- |
| MB638028           | ABILENE BEEHIVE I... | 442 CEDAR ST STE ... | ABILENE              | TX             | 79601             | ... |
| MB543114           | ABILENE BOWLING L... | 279 RUIDOSA AVE  ... | ABILENE              | TX             | 79605             | ... |
| MB933130           | ABILENE CABARET L... | 1918 BUTTERNUT ST... | ABILENE              | TX             | 79602             | ... |
| N 037863           | ABILENE COUNTRY C... | 4039 S TREADAWAY ... | ABILENE              | TX             | 79602             | ... |
| MB200506           | ABILENE SEAFOOD T... | 1882 S CLACK ST  ... | ABILENE              | TX             | 79605             | ... |


In [47]:
new_table = mixbev.compute([
    ('name_stripped', agate.Formula(text_type, lambda r: r['Trade Name'].strip()))
])

NameError: name 'text_type' is not defined

In [48]:
new_table.print_table()

NameError: name 'new_table' is not defined